First, import needed modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
import os

Initalize variables

In [2]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [133]:
btw = pd.read_csv(f'{path}/data/btw_treat.csv', encoding = 'UTF-8-SIG', sep=',')
#data = data.set_index(['AGS', 'year'])
btw = btw[btw["vote_type"] == 'office']

Split data in Erst- and Zweitstimme

In [134]:
erst = btw[btw['first_vote'] == 1]
zweit = btw[btw['second_vote'] == 1]

In [135]:
erst.columns

Index(['AGS', 'GEN', 'year', 'Land', 'state_id', 'post_2005', 'post_2009',
       'post_2013', 'post_2017', 'first_vote', 'second_vote', 'vote_type',
       'Wähler', 'Gültig', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere',
       'treatment', 'treated_0', 'treated_10', 'treated_20', 'treated_30',
       'treated_50', 'treated_100'],
      dtype='object')

Statsmodels

Define a function for regression models

In [144]:
from unittest import result


def regression(post, dataset, vote_type):
    tables = []
    results = {}
    treatment_dummies = ['treated_0', 'treated_10', 'treated_20', 'treated_30', 'treated_50', 'treated_100']
    parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
    post_dummies = {'post_2005': 'between 2002 and 2005', 'post_2009': 'between 2005 and 2009', 'post_2013': 'between 2009 and 2013', 'post_2017': 'between 2013 and 2017'}
    for treatment in treatment_dummies:
        for party in parties:
            results[party] = smf.ols(f'{party} ~ {treatment}*{post}', data=dataset).fit(cov_type='cluster', cov_kwds={'groups': np.array(dataset['AGS'])}, missing='drop')
        stargazer = Stargazer([results['Union'], results['SPD'], results['FDP'], results['Linke'], results['Grüne'], results['Andere']])
        distance = treatment.strip('treated_')
        #stargazer.title(f'Effect on Primary Vote, Municipalities within {distance} km of a powerline project, treatment between 2013 and 2017')
        stargazer.title(f'Effect on {vote_type} Vote, treatment {post_dummies[post]}')
        stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
        stargazer.significant_digits(3)
        #stargazer.covariate_order(covar_order)
        #stargazer.rename_covariates(covar_names)
        stargazer.show_model_numbers(False)
        #stargazer.add_custom_notes(['First note', 'Second note'])
        #stargazer.add_line('State dummies', ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', ])
        table = stargazer.render_latex()
        tables.append(table)
    return tables

Define a function to export tables in one .tex dok

In [150]:
def tab_to_tex(name, tables):
    table_parts = []
    with open(f'{path}/tables/btw/{name}.tex', 'w') as f:
        header = r'\documentclass[12pt]{article}' + '\n' + r'\usepackage[a1paper]{geometry}' + '\n' + r'\begin{document}' + '\n \n'
        f.write(header)
        for index, table in enumerate(tables, start=1): 
            table = table.replace('t_', 't')
            table = table.replace('d_', 'd')
            table = table.replace('0:p', '0*p')
            lines = r'\hline \\[-1.8ex]'
            table_parts = table.split(lines)
            if index == 1:
                f.write(table_parts[0] + lines +  table_parts[1] + lines + table_parts[2] + lines)
            elif index < len(tables):
                f.write(table_parts[2] + lines)
            elif index == len(tables):
                f.write(table_parts[2] + lines + table_parts[3] + lines + table_parts[4])
        closer = '\n' + r'\end{document}'
        f.write(closer)

DiD for Treatment between 2013 and 2017

In [149]:
post = 'post_2017'
# primary vote
erst17_tables = regression(post, erst, 'Primary')
tab_to_tex('2017_erst', erst17_tables)
# secondary vote 
zweit17_tables = regression(post, zweit, 'Secondary')
tab_to_tex('2017_zweit', zweit17_tables)

Placebo DiD for treatments between 2005 and 2009

In [146]:
# Drop all observations after the treatment
erst_pl = erst[erst['year'] <= 2013]
zweit_pl = zweit[zweit['year'] <= 2013]

In [147]:
post = 'post_2009'
erst09_tables = regression(post, erst_pl, 'Primary')
tab_to_tex('2009_erst', erst09_tables)
zweit09_tables = regression(post, zweit_pl, 'Secondary')
tab_to_tex('2009_zweit', zweit09_tables)

Placebo DiD for treatments between 2005 and 2009 with controls and fixed effects